In [6]:
import numpy as np
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

#llm
llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)
# Two prompts
physics_template = """
You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """
You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}
"""

#embed prompt
embedding_moedl=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
physics_prompt_embedding=embedding_moedl.embed_query(physics_template)
math_prompt_embedding=embedding_moedl.embed_query(math_template)


#cosine_similarity
def cosine_similarity(vec1, vec2):
    dot = np.dot(vec1, vec2)
    return dot / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

#Route question to prompt

def prompt_router(input):
    #embed question
    query_embedding=embedding_moedl.embed_query(input["query"])
    # Compute similarityte
    physics_similarity=cosine_similarity(query_embedding,physics_prompt_embedding)
    math_similarity=cosine_similarity(query_embedding,math_prompt_embedding)

    if physics_similarity > math_similarity:
        print("using physics")
        return PromptTemplate.from_template(physics_template)
    else:
        print("using math")
        return PromptTemplate.from_template(math_template)

chain=(
    {"query":RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)
chain.invoke("What is atom")

using physics


"A fundamental question!\n\nAn atom is the smallest unit of ordinary matter that forms a chemical element. It's made up of three main parts: protons, neutrons, and electrons.\n\n* Protons and neutrons are found in the nucleus (center) of the atom, and they have a positive charge.\n* Electrons orbit around the nucleus, and they have a negative charge.\n* The number of protons in an atom determines the element it represents. For example, hydrogen has one proton, helium has two, and so on.\n\nThink of an atom like a tiny solar system, with the nucleus as the sun and the electrons as planets orbiting around it!\n\nDoes that make sense?"